# IBM Data Science Specialization Course 9

This notebook will be mainly used for the capstone project.

In [1]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Course 9 week 3 assignment: Toronto

Scrape the Wikipedia table at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M (retrieved 24 Feb 2019) into a pandas DataFrame.

## First try: parse Wikipedia using py-wikimarkup

Initial how-to instructions from https://stackoverflow.com/questions/15724034/how-to-convert-wikipedia-wikitable-to-python-pandas-dataframe (retrieved 24 Feb 2019).

In [2]:
!pip install py-wikimarkup
!pip install pyquery

  Running setup.py bdist_wheel for py-wikimarkup ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/50/a3/eb/47f4014d6bd8d2d233b552150d8ba308299cdf0bd2046ddd91
Successfully built py-wikimarkup
Requirement not upgraded as not directly required: lxml>=2.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyquery)


In [3]:
import wikimarkup
import pandas as pd
from pyquery import PyQuery

def get_tables(wiki):
    html = PyQuery(wikimarkup.parse(wiki))
    frames = []
    for table in html('table'):
        data = [[x.text.strip() for x in row]
                for row in table.getchildren()]
        df = pd.DataFrame(data[1:], columns=data[0])
        frames.append(df)
    return frames


ImportError: cannot import name 'parse'

Unfortunately, a pre-installed "parse" package in IBM Watson Studio conflicts with the "parse.py" file name inside wikimarkup.

## Second try: parse Wikipedia using BeautifulSoup

Following https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722 (retrieved 24 Feb 2019)

Gettin HTML of the Wikipedia page:

In [7]:
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')
print("{}\n\n   [...]\n\n{}".format(soup.prettify()[:500], soup.prettify()[-500:]))

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_po

   [...]

":"Contributors to Wikimedia projects"},"publisher":{"@type":"Organization","name":"Wikimedia Foundation, Inc.","logo":{"@type":"ImageObject","url":"https:\/\/www.wikimedia.org\/static\/images\/wmf-hor-googpub.png"}},"datePublished":"2004-03-20T10:02:13Z","dateModified":"2019-01-04T18:32:45Z","headline":"Wikimedia list article"}
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgBackendResponseTime":99,"wgHostname":"mw1251"});});
  </script>
 </bo

This library seems to function OK.

The needed data is contained in the first (and only) table, and is rendered with HTML style "wikitable sortable". Parse it:

In [8]:
My_table = soup.find('table', { 'class' : 'wikitable sortable' })
print("{}\n\n   [...]\n\n{}".format(str(My_table)[:500], str(My_table)[-500:]))

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>

   [...]

/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Queensway" title="The Queensway">The Queensway West</a>
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Royal York South West
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Bloor" title="Bloor">South of Bloor</a>
</td></tr>
<tr>
<td>M9Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
</tb

## Clean data for postal code, borough, and neighborhood

Now loop through all rows from that table. For each row:
1. Ignore if it is the header row (no "TD" tags),
2. strip leading and trailing whitespace (including line breaks),
3. Ignore if the borough is "Not assigned",
4. If the borough is given but neighborhoos is "Not assigned", fill in the borough as neighborhood name.
Finally, build a new pandas DataFrame from the list items collected.

In [22]:
l = []
unassigned_pc = []

table_rows = My_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("ignoring empty row")
    elif row[1] == 'Not assigned':
        unassigned_pc.append(row[0])
    else:
        if row[2] == 'Not assigned':
            row[2] = row[1]
            print('assigning neighborhood same as borough: {}'.format(row[2]))
        l.append(row)

print("ignoring unassigned postal codes:{}".format(unassigned_pc))
        
My_df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"])
print("My_df.head()=\n{}".format(My_df.head()))
print("total: My_df.shape={}".format(My_df.shape))

ignoring empty row
assigning neighborhood same as borough: Queen's Park
ignoring unassigned postal codes:['M1A', 'M2A', 'M8A', 'M2B', 'M7B', 'M8B', 'M2C', 'M7C', 'M8C', 'M2E', 'M3E', 'M7E', 'M8E', 'M9E', 'M2G', 'M3G', 'M7G', 'M8G', 'M9G', 'M7H', 'M8H', 'M9H', 'M7J', 'M8J', 'M9J', 'M7K', 'M8K', 'M9K', 'M7L', 'M8L', 'M7M', 'M8M', 'M7N', 'M8N', 'M3P', 'M7P', 'M8P', 'M3R', 'M8R', 'M2S', 'M3S', 'M7S', 'M8S', 'M9S', 'M2T', 'M3T', 'M6T', 'M7T', 'M8T', 'M9T', 'M2V', 'M3V', 'M6V', 'M7V', 'M2W', 'M3W', 'M6W', 'M7W', 'M2X', 'M3X', 'M6X', 'M7X', 'M9X', 'M1Y', 'M2Y', 'M3Y', 'M5Y', 'M6Y', 'M9Y', 'M1Z', 'M2Z', 'M3Z', 'M4Z', 'M5Z', 'M6Z', 'M7Z', 'M9Z']
My_df.head()=
  PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights
total: My_df.shape=(212, 3)


## Aggregate neighborhoods by borough and postal code

Concatenate neighborhoods grouped by postal code and borough.

In [23]:
df_grouped = My_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
print("df_grouped.head()=\n{}".format(df_grouped.head()))
print("total: df_grouped.shape={}".format(df_grouped.shape))

df_grouped.head()=
  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Rouge, Malvern
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae
total: df_grouped.shape=(103, 3)


## Result: Number of rows in Toronto neighborhood DataFrame

In [25]:
print("Number of records in cleaned, aggregated Toronto neighborhoods scraped from Wikipedia:\n{}".format(df_grouped.shape[0]))

Number of records in cleaned, aggregated Toronto neighborhoods scraped from Wikipedia:
103


## Geolocation of Toronto neighborhoods

Install geocoder library and try to get location for the neighborhoods

In [26]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 6.8MB/s ta 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda

In [33]:
import geocoder # import geocoder

# testing geocoder
lat_lng_coords = None
cnt = 0

# loop until you get the coordinates
while(lat_lng_coords is None and cnt < 10):
    print('attempt #{}...'.format(cnt))
    cnt += 1
    g = geocoder.google('Mountain View, CA')
    lat_lng_coords = g.latlng

if lat_lng_coords is None:
    print('All API calls unsuccessful')
else:
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print("Success, lat/long = {}/{}".format(latitude, longitude))


attempt #0...
attempt #1...
attempt #2...
attempt #3...
attempt #4...
attempt #5...
attempt #6...
attempt #7...
attempt #8...
attempt #9...
All API calls unsuccessful


Google doesn't seem to work well, try OpenStreetMap

In [40]:
osm_result = None
cnt = 0

# loop until you get the coordinates
while(osm_result is None and cnt < 10):
    print('attempt #{}...'.format(cnt))
    cnt += 1
    g = geocoder.osm('M1B, Toronto, Ontario, Canada')
    osm_result = g.json

if osm_result is None:
    print('All API calls unsuccessful')
else:
    print("Success:\n{}".format(g.json))

attempt #0...
Success:
{'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_place_city.p.20.png', 'osm_type': 'node', 'ok': True, 'place_id': '110816', 'importance': 1.066077038952242, 'lat': 43.653963, 'region': 'Ontario', 'country_code': 'ca', 'osm_id': '18063533', 'bbox': {'northeast': [43.813963, -79.227207], 'southwest': [43.493963, -79.547207]}, 'quality': 'city', 'state': 'Ontario', 'postal': 'M6K 1X9', 'address': 'Toronto, Ontario, M6K 1X9, Canada', 'place_rank': '16', 'type': 'city', 'accuracy': 1.066077038952242, 'raw': {'lon': '-79.387207', 'lat': '43.653963', 'osm_type': 'node', 'display_name': 'Toronto, Ontario, M6K 1X9, Canada', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'place_id': '110816', 'importance': 1.066077038952242, 'category': 'place', 'boundingbox': ['43.493963', '43.813963', '-79.547207', '-79.227207'], 'osm_id': '18063533', 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_place_city.p.20.p

This seems to work on first try.

Write a function and see how it performs on all postal codes:

In [59]:
def get_osm_loc(param='', base_address='Toronto, Ontario, {}, Canada'):
    g = geocoder.osm(base_address.format(param))
    osm_result = g.json
    if osm_result is None:
        # API call unsuccessful
        return 0, 0
    else:
        # success, extract lat/long
        latitude = osm_result['lat']
        longitude = osm_result['lng']
        return (latitude, longitude)

print('get_osm_loc("", "Paris, France")={}'.format(get_osm_loc("", "Paris, France")))

for test_pc in ['M5G', 'M1B', 'M2H', 'M4B']:
    print('get_osm_loc("{}")={}'.format(test_pc, get_osm_loc(test_pc)))


get_osm_loc("", "Paris, France")=(48.8566101, 2.3514992)
get_osm_loc("M5G")=(0, 0)
get_osm_loc("M1B")=(43.653963, -79.387207)
get_osm_loc("M2H")=(0, 0)
get_osm_loc("M4B")=(0, 0)


Ok, OpenStreenMap resolves some but not all locations (including postal codes in Toronto).

## Try geocoder with CSV lookup as fallback

Loop through all records obtained earlier and try to get geolocation dynamically for the Toronto boroughs. After 10 tries, fall back to using a provided CSV file as lookup.

In [70]:
geo_csv_url = 'https://cocl.us/Geospatial_data'
geo_df = pd.read_csv(geo_csv_url)
print('geo_df.head()=\n{}'.format(geo_df.head()))


geo_df.head()=
  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [99]:
def get_pc_loc(pc):
    (latitude, longitude) = get_osm_loc(pc)
    if latitude == 0 and longitude == 0:
        loc_info = geo_df[geo_df['Postal Code'] == pc]
        if loc_info.shape[0] > 0:
            latitude = loc_info['Latitude'].values[0]
            longitude = loc_info['Longitude'].values[0]
            print("{}: got lat/long from CSV".format(pc))
        else:
            latitude = 0
            longitude = 0
            print("{}: could not find lat/long".format(pc))
    else:
        print("{}: got lat/long from osm".format(pc))
    return (latitude, longitude)

for test_pc in ['M5G', 'M1B', 'M2H', 'M4B']:
    print('get_pc_loc("{}")={}'.format(test_pc, get_pc_loc(test_pc)))


M5G: got lat/long from CSV
get_pc_loc("M5G")=(43.657952399999999, -79.387382599999995)
M1B: got lat/long from osm
get_pc_loc("M1B")=(43.653963, -79.387207)
M2H: got lat/long from CSV
get_pc_loc("M2H")=(43.803762200000001, -79.363451699999999)
M4B: got lat/long from CSV
get_pc_loc("M4B")=(43.706397199999998, -79.309937000000005)


This looks good, ready to put it all together.

For every row in the DataFrame that is grouped by Postal Code and Borough, apply the get_pc_local() function and put the lat/long values into a
new temporary `TempLatLong` column.

In [101]:
print("Obtaining latitudes...")
df_grouped['TempLatLong'] = df_grouped.apply(lambda x : get_pc_loc(x['PostalCode']), axis=1)


Obtaining latitudes...
M1B: got lat/long from osm
M1C: got lat/long from osm
M1E: got lat/long from CSV
M1G: got lat/long from osm
M1H: got lat/long from CSV
M1J: got lat/long from CSV
M1K: got lat/long from CSV
M1L: got lat/long from CSV
M1M: got lat/long from CSV
M1N: got lat/long from CSV
M1P: got lat/long from CSV
M1R: got lat/long from CSV
M1S: got lat/long from CSV
M1T: got lat/long from CSV
M1V: got lat/long from CSV
M1W: got lat/long from osm
M1X: got lat/long from CSV
M2H: got lat/long from CSV
M2J: got lat/long from osm
M2K: got lat/long from CSV
M2L: got lat/long from CSV
M2M: got lat/long from CSV
M2N: got lat/long from osm
M2P: got lat/long from CSV
M2R: got lat/long from CSV
M3A: got lat/long from CSV
M3B: got lat/long from CSV
M3C: got lat/long from osm
M3H: got lat/long from osm
M3J: got lat/long from osm
M3K: got lat/long from CSV
M3L: got lat/long from CSV
M3M: got lat/long from CSV
M3N: got lat/long from CSV
M4A: got lat/long from CSV
M4B: got lat/long from CSV
M4C: 

In [105]:
print('The first 10 records of the new DataFrame:')
df_grouped.head(10)

The first 10 records of the new DataFrame:


,PostalCode,Borough,Neighborhood,Latitude,Longitude,TempLatLong
0,M1B,Scarborough,"Rouge, Malvern",NaN,NaN,"(43.653963, -79.387207)"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",NaN,NaN,"(43.653963, -79.387207)"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN,"(43.7635726, -79.1887115)"
3,M1G,Scarborough,Woburn,NaN,NaN,"(43.7394839, -79.369314)"
4,M1H,Scarborough,Cedarbrae,NaN,NaN,"(43.773136, -79.2394761)"
5,M1J,Scarborough,Scarborough Village,NaN,NaN,"(43.7447342, -79.2394761)"
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",NaN,NaN,"(43.7279292, -79.2620294)"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",NaN,NaN,"(43.7111117, -79.2845772)"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",NaN,NaN,"(43.716316, -79.2394761)"
9,M1N,Scarborough,"Birch Cliff, Cliffside West",NaN,NaN,"(43.692657, -79.2648481)"


Extract the lat/long coordinates from the `TempLatLong` column, and create the DataFrame in the format required by the assignment.

In [113]:
df_grouped['Latitude'] = df_grouped.apply(lambda x : x['TempLatLong'][0], axis=1)
df_grouped['Longitude'] = df_grouped.apply(lambda x : x['TempLatLong'][1], axis=1)
df_grouped = df_grouped.drop(['TempLatLong'], axis=1)

In [114]:
print('The first 10 records of the new DataFrame:')
df_grouped.head(10)

The first 10 records of the new DataFrame:


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.653963,-79.387207
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.653963,-79.387207
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.739484,-79.369314
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Explore neighborhoods

Limit to boroughs that contain the word "Toronto", then perform the same analysis as we did on the New York City data set.

In [121]:
print('All boroughs: {}'.format(df_grouped['Borough'].unique()))
df_toronto = df_grouped[df_grouped['Borough'].str.contains('Toronto')]
print('Boroughs with "Toronto" in the name: {}'.format(df_toronto['Borough'].unique()))

All boroughs: ['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']
Boroughs with "Toronto" in the name: ['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


In [123]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.91 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.63 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.11 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.41 MB/s
vincent-0.4.4- 100% |###################

In [136]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.68,-79.37], zoom_start=13)

neighborhoods = pd.DataFrame(df_toronto)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


In [185]:
# Copy credentials here
CLIENT_ID = ''
CLIENT_SECRET = ''


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


(this cell holds the authentication details)

<!--

CLIENT_ID = 'NBLOR5JJCSM43LTXYWBQYVJ5U3LMNZ2ULCHERZAZVLJTHBYA' # your Foursquare ID
CLIENT_SECRET = 'D1G4RELNK2MGSOZSO1C4DTGPYBAWHIW0MQJTXWSBTGH2JL41' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

-->

In [145]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [146]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



print(toronto_venues.shape)
toronto_venues.head()


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [147]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,71,71,71,71,71,71
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",48,48,48,48,48,48
Central Bay Street,81,81,81,81,81,81
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [149]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 232 uniques categories.


In [150]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Yoga Studio,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
toronto_onehot.shape

(1911, 232)

In [152]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,...,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.00,0.040000,0.000000,0.00,0.010000,0.010000,...,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.00,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.00,0.028169,0.000000,0.00,0.056338,0.014085,...,0.000000,0.00,0.014085,0.014085,0.000000,0.000000,0.000000,0.00,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.030000,0.000000,0.000000,0.01,0.020000,0.000000,0.01,0.000000,0.000000,...,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.01
3,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,...,0.000000,0.00,0.000000,0.030000,0.010000,0.000000,0.000000,0.00,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
6,Central Bay Street,0.012346,0.000000,0.000000,0.00,0.012346,0.000000,0.00,0.000000,0.012346,...,0.000000,0.00,0.000000,0.012346,0.000000,0.000000,0.012346,0.00,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.000000,...,0.000000,0.00,0.000000,0.050000,0.000000,0.050000,0.010000,0.00,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
9,Church and Wellesley,0.011628,0.011628,0.011628,0.00,0.011628,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.011628,0.011628,0.000000,0.00,0.011628,0.00


In [153]:
toronto_grouped.shape

(38, 232)

In [154]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
                 venue  freq
0                 Café  0.07
1          Art Gallery  0.06
2          Coffee Shop  0.06
3       Breakfast Spot  0.04
4  Japanese Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0          Restaurant  0.06
1         Coffee Shop  0.05
2  Italian Restaurant  0.05
3                 Spa  0.04
4                 Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.05
1     Garden Center  0.05
2              Park  0.05
3  Recording Studio  0.05
4        Restaurant  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Cof

In [155]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [157]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Asian Restaurant,Restaurant,Bar,Gym,Hotel
1,Berczy Park,Café,Art Gallery,Coffee Shop,Breakfast Spot,Japanese Restaurant,American Restaurant,Bar,Chinese Restaurant,Sushi Restaurant,Exhibit
2,"Brockton, Exhibition Place, Parkdale Village",Restaurant,Italian Restaurant,Coffee Shop,Bar,Spa,Yoga Studio,Café,Furniture / Home Store,Bakery,Gym
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Comic Shop,Smoke Shop,Skate Park,Burrito Place,Butcher,Restaurant,Recording Studio,Park,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Restaurant,Italian Restaurant,Sandwich Place,Hotel,French Restaurant,Vegetarian / Vegan Restaurant,Steakhouse,Café,Pizza Place


In [158]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [173]:
# add clustering labels

if 'Cluster Labels' in neighborhoods_venues_sorted.columns:
    neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop('Cluster Labels', axis=1)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Coffee Shop,Women's Store,Donut Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Dessert Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Sandwich Place,Pizza Place,Italian Restaurant,Pet Store,Pub,Movie Theater,Burrito Place,Burger Joint,Fast Food Restaurant,Brewery
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Brewery,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Dim Sum Restaurant,Swim School,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market


In [176]:
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.68,-79.37], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [177]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Health Food Store,Pub,Coffee Shop,Women's Store,Donut Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
41,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Dessert Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant
42,East Toronto,0,Sandwich Place,Pizza Place,Italian Restaurant,Pet Store,Pub,Movie Theater,Burrito Place,Burger Joint,Fast Food Restaurant,Brewery
43,East Toronto,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Brewery,Seafood Restaurant,Sandwich Place
45,Central Toronto,0,Clothing Store,Gym,Food & Drink Shop,Burger Joint,Sandwich Place,Grocery Store,Breakfast Spot,Hotel,Park,Department Store
46,Central Toronto,0,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Furniture / Home Store,Pet Store,Park,Dessert Shop,Diner,Rental Car Location
47,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Café,Restaurant,Sushi Restaurant,Thai Restaurant,Italian Restaurant,Coffee Shop,Chinese Restaurant
49,Central Toronto,0,Convenience Store,Pub,Coffee Shop,Supermarket,Vietnamese Restaurant,Pizza Place,American Restaurant,Bagel Shop,Fried Chicken Joint,Sushi Restaurant
51,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Pizza Place,Italian Restaurant,Bakery,Pub,Park,Pet Store,Pharmacy
52,Downtown Toronto,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Pub,Men's Store,Mediterranean Restaurant,Café


In [179]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,1,Park,Playground,Trail,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit


In [180]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,2,Playground,Gym,Restaurant,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit


In [181]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Home Service,Garden,Music Venue,Dumpling Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit


In [182]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Park,Bus Line,Dim Sum Restaurant,Swim School,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
84,West Toronto,4,Park,Gym / Fitness Center,Light Rail Station,Building,Beach,Bus Stop,Trail,American Restaurant,Lake,Fast Food Restaurant
